# Pressure waves in a box

In this lesson we examine the 2D wave equation simulating a pressure wave traversing through a rectangular box. This simulation relates closely to what occurs inside of a piezoelectrically driven printhead. In such devices a piezoelectric transducer periodically causes mechanical deformation of an ink channel, inputting pressure waves into to fluid which reflect around and intefere with eachother resulting in the despensing of droplets. See a simplified schematic below.

<img src="inkchannel.png">
<br/>
#### Figure 1. Piezoelecric inkjet printhead schematic

### The 2D wave Equation

The wave equation is a hyperbolic partial differential equation. In 2D it is written as follows:

\begin{equation}
\frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u
\end{equation}

Or writing out the Laplacian, $\nabla^2$

\begin{equation}
\frac{\partial^2 u}{\partial t^2} = c^2\left(\frac{\partial^2 u}{\partial x^2} + \
\frac{\partial^2 u}{\partial y^2}\right)
\end{equation}

### Discretization
Here we are going to solve it using a central time, central space leapfrog method. First we need to descretize the equation. We will denote time steps by the letter $n$, spatial steps in the x and y directions will be denoted by $i$ and $j$ respectively. Applying the central difference in time and space the the left and righ hand sides gives:

\begin{equation}
\frac{u_{ij}^{n+1} - 2u_{ij}^{n} + u_{ij}^{n-1}}{\Delta t^{2}} =
c^2\left(\frac{u^{n}_{i+1,j} - 2u^{n}_{ij} + u^{n}_{i-1,j}}{\Delta x^{2}} +\
\frac{u^{n}_{i,j+1} - 2u^{n}_{ij} + u^{n}_{i,j-1}}{\Delta y^{2}}\right)
\end{equation}

<br/>
We then need to solve for the unknown variable $u_{ij}^{n+1}$
<br/>

Letting $\sigma_x = \frac{\Delta t^2 c^2}{\Delta x^2}$ and $\sigma_y = \frac{\Delta t^2 c^2}{\Delta y^2}$ the expression for u at the next time step becomes:

\begin{equation}
u_{ij}^{n+1} = \sigma_x^2(u_{i+1,j}^n + u_{i-1,j}^n) + \sigma_y^2(u_{i,j+1}^n + \
u_{i,j-1}^n) + 2(1-\sigma_x - \sigma_y)u_{ij}^n - u_{ij}^n
\end{equation}

Great, now we need to deal with the boundary conditions.

### Boundary Conditions 
We want the wave to be reflected back into the domain at the boundaries. This can be done with either Dirichlet or Neumann boundary conditions. Explained in more detail [here](http://hplgit.github.io/INF5620/doc/pub/sphinx-wave/._main_wave003.html
).

We will be applying the Neumann boundary conditions so to preserve the sign of the reflected wave. The boundary conditions are defined using the normal derivitave of u at the boundaries:

\begin{equation}
\frac{\partial n}{\partial n}\equiv \textbf{n} \cdot\nabla u = 0
\end{equation}

The derivative with respect to n is in the outward normal direction from the boundary. 

To imcorporate this condition into our scheme we apply the same sentral difference approximation. At the left hand vertical boundary where $i = 0$ we have

\begin{equation}
\frac{u_{-1,j}^n - u_{1,j}^n}{2\Delta x} = 0
\end{equation}

however, the value $u_{-1,j}$ is not defined in our domain. We can either make use of an extra layer of points all around our domain (ghost points), or 

In [1]:
from IPython.core.display import HTML
css_file = 'numericalmoocstyle.css'
HTML(open(css_file, "r").read())